<a href="https://colab.research.google.com/github/lynnfdsouza/drone_terrain_masking/blob/main/real_time_updates_drone_exec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import time

class QuantumIMU:
    def __init__(self, imu_api):
        self.imu_api = imu_api

    def read_acceleration(self):
        # Get acceleration (m/s^2) from quantum accelerometer
        return self.imu_api.get_acceleration()

    def read_gyroscope(self):
        # Get rotation rate (deg/s or rad/s) from quantum gyroscope
        return self.imu_api.get_gyroscope()

class QuantumMagnetometer:
    def __init__(self, mag_api):
        self.mag_api = mag_api

    def read_magnetic_field(self):
        # Get magnetic field (Tesla) from quantum magnetometer
        return self.mag_api.get_magnetic_field()

class QuantumGravimeter:
    def __init__(self, grav_api):
        self.grav_api = grav_api

    def read_gravity(self):
        # Get local gravity vector for terrain following
        return self.grav_api.get_gravity()

class TerrainMaskingNavigator:
    def __init__(self, imu, mag, gravimeter=None):
        self.imu = imu
        self.mag = mag
        self.grav = gravimeter
        self.position = np.zeros(3)
        self.velocity = np.zeros(3)
        self.orientation = R.from_matrix(np.eye(3))  # Use scipy Rotation object

    def update(self, dt):
        accel = self.imu.read_acceleration()
        gyro = self.imu.read_gyroscope()
        mag_field = self.mag.read_magnetic_field()
        gravity = self.grav.read_gravity() if self.grav else np.array([0, 0, 9.81])

        # Orientation update using quaternions
        self.orientation = self.update_orientation(self.orientation, gyro, dt)

        # Velocity update
        acc_world = self.orientation.apply(accel) - gravity  # Apply rotation using scipy
        self.velocity += acc_world * dt

        # Position update
        self.position += self.velocity * dt

        # Terrain masking logic (pseudo)
        terrain_elevation = self.get_terrain_elevation(self.position)
        if self.position[2] < terrain_elevation + 10:  # Keep 10m above terrain
            self.position[2] = terrain_elevation + 10

        return self.position, self.velocity, self.orientation.as_matrix() # Return as matrix for now

    def update_orientation(self, orientation, gyro, dt):
        # Integrate gyro to update orientation using quaternions
        # Convert gyro rate to quaternion rate
        gyro_quat = R.from_rotvec(gyro * dt)
        # Update orientation
        new_orientation = orientation * gyro_quat
        return new_orientation

    def get_terrain_elevation(self, position):
        # Placeholder for DEM (Digital Elevation Model) lookup
        # Mock implementation: varying elevation based on x and y position
        x, y, z = position
        # Create a simple hilly terrain
        elevation = 10 * np.sin(x/100) + 15 * np.cos(y/100)
        return elevation

# Mock API classes for demonstration
class MockIMUAPI:
    def get_acceleration(self):
        return np.array([0.1, 0.2, 9.8]) # Example acceleration

    def get_gyroscope(self):
        return np.array([0.01, 0.02, 0.03]) # Example gyro data

class MockMagnetometerAPI:
    def get_magnetic_field(self):
        return np.array([50e-6, 20e-6, -30e-6]) # Example magnetic field

class MockGravimeterAPI:
    def get_gravity(self):
        return np.array([0, 0, 9.81]) # Example gravity vector


# Example usage (you would replace api classes with actual device interfaces)
imu_api = MockIMUAPI()  # Replace with actual quantum IMU API
mag_api = MockMagnetometerAPI()  # Replace with actual quantum magnetometer API
grav_api = MockGravimeterAPI() # Replace with actual quantum gravimeter API

imu = QuantumIMU(imu_api)
mag = QuantumMagnetometer(mag_api)
grav = QuantumGravimeter(grav_api)

navigator = TerrainMaskingNavigator(imu, mag, grav)

# Run the simulation for a few steps
# num_steps = 10
dt = 0.01
# for i in range(num_steps):
#     pos, vel, ori = navigator.update(dt)
#     print(f"Step {i+1}: Position: {pos}, Velocity: {vel}")

while True:
    pos, vel, ori = navigator.update(dt=dt)
    print(f"Position: {pos}, Velocity: {vel}")
    # Add drone control logic here
    # Example: Simple control to maintain a constant altitude above terrain
    terrain_elevation = navigator.get_terrain_elevation(pos)
    altitude_error = (pos[2] - terrain_elevation) - 10 # Desired altitude is 10m above terrain

    # Simple proportional control for vertical velocity
    vertical_command = -0.5 * altitude_error # Adjust vertical velocity to reduce altitude error
    # You would then translate this command into motor controls for a real drone

    # You might also add control logic for horizontal movement,
    # heading, etc. based on mission objectives or external inputs.

    # Add a small delay to simulate real-time updates
    time.sleep(dt)

Streaming output truncated to the last 5000 lines.
Position: [5.91697879e+04 6.09848126e+04 3.27469965e+01], Velocity: [ 389.09742325  711.99278203 -575.3933291 ]
Position: [5.91736793e+04 6.09919325e+04 3.24939510e+01], Velocity: [ 389.13253996  711.98765963 -575.40005308]
Position: [5.91775710e+04 6.09990524e+04 3.21588844e+01], Velocity: [ 389.16767647  711.98253864 -575.40678459]
Position: [5.91814630e+04 6.10061721e+04 3.17440734e+01], Velocity: [ 389.2028328   711.97741906 -575.41352364]
Position: [5.91853554e+04 6.10132918e+04 3.12521496e+01], Velocity: [ 389.23800893  711.97230089 -575.42027024]
Position: [5.91892481e+04 6.10204115e+04 3.06860850e+01], Velocity: [ 389.27320487  711.96718414 -575.42702439]
Position: [5.91931412e+04 6.10275311e+04 3.00491763e+01], Velocity: [ 389.30842061  711.96206882 -575.43378609]
Position: [5.91970346e+04 6.10346507e+04 2.93450275e+01], Velocity: [ 389.34365616  711.95695494 -575.44055534]
Position: [5.92009284e+04 6.10417702e+04 2.85775311e+

KeyboardInterrupt: 